In [1]:
import pandas as pd
import numpy as np
from math import log2
from collections import Counter
import glob
import os

In [2]:
# Define paths
path = "/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze"
save_matrix_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/Matrices/"
entropy_log_path = "/Volumes/TwoTeras/1_Experiment_2/Entropy_Results/entropy_log.csv"

# Ensure directories exist
os.makedirs(save_matrix_path, exist_ok=True)

# Load all CSV files in the path
files = glob.glob(path + "/*.csv")

# Mapping dictionary
mapping = {
    "TaskBuilding_Public": "Task_Building",
    "TaskBuilding_Residential": "Task_Building",
    "Active_Agent": "Agent_Body",
    "Passive_Agent": "Agent_Body",
    "Active_Agent_Face": "Agent_Face",
    "Passive_Agent_Face": "Agent_Face"
}

# Initialize list to store entropy results
entropy_results = []
failed_files = []

# Function to calculate conditional entropy H(X|Y=y) for each row
def calculate_transition_entropy(matrix, stationary_distribution):
    total_entropy = 0
    category_entropies = {}
    for i, row in matrix.iterrows():
        row_entropy = sum(-p * log2(p) for p in row if p > 0)
        category_entropies[i] = row_entropy
        total_entropy += row_entropy * stationary_distribution.get(i, 0)
    return total_entropy, category_entropies

# Loop through all files
for filename in files:
    try:
        print(f"Processing file: {filename}")  # Debugging statement

        # Read participant data
        One_participant = pd.read_csv(filename)
        if 'timeStampDataPointEnd' not in One_participant.columns or 'Collider_CategoricalN' not in One_participant.columns:
            print(f"Skipping {filename} due to missing columns.")
            failed_files.append({'filename': filename[-10:-4], 'error': 'Missing required columns'})
            continue

        # Convert timestamps (if required)
        try:
            One_participant['date_seconds'] = pd.to_datetime(One_participant['timeStampDataPointEnd'], unit='s')
        except Exception as e:
            print(f"Timestamp conversion error in {filename}: {e}")
            failed_files.append({'filename': filename[-10:-4], 'error': str(e)})
            continue

        # Filter for the desired gaze events
        data_Reduced = One_participant[One_participant['events'] == -2]
        
        # Check if data_Reduced is empty to avoid errors in transition matrix creation
        if data_Reduced.empty:
            print(f"No valid events for {filename[-10:-4]}, skipping.")
            failed_files.append({'filename': filename[-10:-4], 'error': 'No valid events in filtered data'})
            continue

        # Get gaze sequence for category transitions
        # Apply mapping
        ##data_Reduced = data_Reduced.copy()
        #data_Reduced.loc[:, "Mapped_Column_Collider_Categorical"] = data_Reduced["Collider_CategoricalN"].replace(mapping)
        gaze_sequence = data_Reduced['Collider_CategoricalN'].reset_index(drop=True)

        # Create transition matrix
        categories = gaze_sequence.unique()
        transition_matrix = pd.DataFrame(0, index=categories, columns=categories, dtype=float)

        # Build the transition matrix by counting transitions
        for i in range(len(gaze_sequence) - 1):
            current_category = gaze_sequence.iloc[i]
            next_category = gaze_sequence.iloc[i + 1]
            transition_matrix.loc[current_category, next_category] += 1

        # Normalize the transition matrix so each row sums to 1 (row normalization only)
        transition_matrix = transition_matrix.div(transition_matrix.sum(axis=1), axis=0).fillna(0)

        # Save the normalized transition matrix
        matrix_filename = os.path.join(save_matrix_path, f"{filename[-10:-4]}_transition_matrix.csv")
        transition_matrix.to_csv(matrix_filename, index=True)
        print(f"Saved transition matrix for {filename[-10:-4]}")  # Debugging statement

        # Calculate the stationary distribution (normalized left eigenvector of eigenvalue 1)
        eigvals, eigvecs = np.linalg.eig(transition_matrix.T)
        stationary_distribution = np.real(eigvecs[:, np.isclose(eigvals, 1)].flatten())
        stationary_distribution /= stationary_distribution.sum()

        # Ensure stationary distribution matches the order of `categories`
        stationary_distribution_dict = {categories[i]: stationary_distribution[i] for i in range(len(categories))}

        # Calculate overall transition entropy and per-category entropies using stationary distribution
        overall_transition_entropy, transition_entropy_per_category = calculate_transition_entropy(transition_matrix, stationary_distribution_dict)

        # Calculate stationary entropy per category (based on stationary distribution values)
        stationary_entropy_per_category = {
            category: (-stationary_distribution_dict[category] * log2(stationary_distribution_dict[category]))
            if stationary_distribution_dict[category] > 0 else 0
            for category in categories
        }

        # Normalize entropies by the number of categories
        num_categories = len(transition_matrix)
        normalized_overall_transition_entropy = overall_transition_entropy / log2(num_categories) if num_categories > 1 else 0

        # Add entropy values to results
        result = {
            'Filename': filename[-10:-4],
            'Overall_Transition_Entropy': normalized_overall_transition_entropy
        }

        # Add each category's transition and stationary entropy to the result
        for category in categories:
            result[f'Transition_Entropy_{category}'] = transition_entropy_per_category.get(category, 0) / log2(num_categories) if num_categories > 1 else 0
            result[f'Stationary_Entropy_{category}'] = stationary_entropy_per_category.get(category, 0) / log2(num_categories) if num_categories > 1 else 0

        entropy_results.append(result)

    except Exception as e:
        # Log any failed files
        print(f"Error processing {filename}: {e}")  # Debugging statement
        failed_files.append({'filename': filename[-10:-4], 'error': str(e)})

# Save entropy results to a CSV
entropy_df = pd.DataFrame(entropy_results)
entropy_df.to_csv(entropy_log_path, index=False)

# Log any failed files if they exist
if failed_files:
    failed_df = pd.DataFrame(failed_files)
    failed_df.to_csv("/Volumes/TwoTeras/1_Experiment_2/Proxemics/failed_files_log_reduced.csv", index=False)
    print(f"Failures logged for {len(failed_files)} files.")
else:
    print("No failures detected.")


Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv
Saved transition matrix for 1031_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_2.csv
Saved transition matrix for 1031_2
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_3.csv
Saved transition matrix for 1031_3
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_4.csv
Saved transition matrix for 1031_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_5.csv
Saved transition matrix for 1031_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1268_1.csv
Saved transition matrix for 1268_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1268_2.csv
Saved transition matrix for 1268_2
Processing file: /Volumes/T

Saved transition matrix for 5161_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_1.csv
Saved transition matrix for 5189_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_2.csv
Saved transition matrix for 5189_2
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_3.csv
Saved transition matrix for 5189_3
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_4.csv
Saved transition matrix for 5189_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5189_5.csv
Saved transition matrix for 5189_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5743_1.csv
Saved transition matrix for 5743_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/5743_2.csv
Saved transition matrix for

Saved transition matrix for 9297_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9297_5.csv
Saved transition matrix for 9297_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_1.csv
Saved transition matrix for 9627_1
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_2.csv
Saved transition matrix for 9627_2
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_3.csv
Saved transition matrix for 9627_3
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_4.csv
Saved transition matrix for 9627_4
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/9627_5.csv
Saved transition matrix for 9627_5
Processing file: /Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1142_1.csv
Saved transition matrix for

In [3]:
data = pd.read_csv("/Volumes/TwoTeras/1_Experiment_2/Eye_Tracking/Pre_processed/05_Debbies_gaze/1031_1.csv")
data['date_seconds'] = pd.to_datetime(data['timeStampDataPointEnd'], unit='s')      

In [4]:
data.names.unique()

array(['pavement_O.002', 'terrain_O.001', 'Wall', 'building01_LOD1',
       'TaskBuilding_27', 'Building_100', 'Building_97',
       'maraz_cafe_collider', 'barbwire0', 'CollisionObject1',
       'Building_161', 'TaskBuilding_35', '23_Cma', 'Graffity_35',
       'Building_166', 'Fence_5', 'road_base_network.004',
       'pavement_Vb.003', 'Building_215', 'Crane_59', 'cyclone0',
       'crane_1', 'Fence_34', 'Building_162', 'Building_99', '20_Cma',
       'Complete_fence.002', 'CollisionObject2', 'Building_94',
       'Building_157', 'building01_LOD0', 'CollisionObject0',
       'terrain_E.001', 'Wall_9', 'Fence_9', 'Fence_10', 'Building_98',
       'Maraz_cafe_place.001', 'Building_148', 'TaskBuilding_5', '05_Cma',
       'pileOfClay_LOD0', 'Building_101', 'Wall_3', 'Body', 'Fence_12',
       'Hedge_9', 'Building_102', 'Lamppost_v1 (15)', 'pavement_E.008',
       'Cypress_v1_2 (1)', 'TaskBuilding_30', 'Graffity_30',
       'Lamppost_v1 (56)', 'Lamppost_v1 (11)', 'road.009', 'Building_1

In [5]:
data_Reduced = data[data['events'] == 2]

In [6]:
data_Reduced[['date_seconds','names', 'Collider_CategoricalN']].head(30)

date_seconds             names     Collider_CategoricalN
1   2022-11-14 14:27:48.847371008    pavement_O.002                Background
4   2022-11-14 14:27:48.901434368    pavement_O.002                Background
32  2022-11-14 14:27:49.403387904              Wall                Background
41  2022-11-14 14:27:49.525403136   building01_LOD1                Background
58  2022-11-14 14:27:49.815069184   TaskBuilding_27       TaskBuilding_Public
89  2022-11-14 14:27:50.229724160      Building_100                  Building
102 2022-11-14 14:27:50.382988288       Building_97                  Building
141 2022-11-14 14:27:50.858653184         barbwire0                Background
203 2022-11-14 14:27:51.735582464      Building_161                  Building
214 2022-11-14 14:27:51.957295104   TaskBuilding_35  TaskBuilding_Residential
225 2022-11-14 14:27:52.179503104   TaskBuilding_35  TaskBuilding_Residential
232 2022-11-14 14:27:52.323840000            23_Cma              Active_Agent
250 2022-11-14 14:27:52.679471104       Graffity_35  TaskBuilding_Residential
270 2022-11-14 14:27:53.035104512       Graffity_35  TaskBuilding_Residential
309 2022-11-14 14:27:53.645184512       Graffity_35  TaskBuilding_Residential
324 2022-11-14 14:27:53.867393536       Graffity_35  TaskBuilding_Residential
333 2022-11-14 14:27:53.978993408       Graffity_35  TaskBuilding_Residential
357 2022-11-14 14:27:54.578657536      Building_166                  Building
362 2022-11-14 14:27:54.656529408      Building_166                  Building
375 2022-11-14 14:27:54.878738432           Fence_5                Background
390 2022-11-14 14:27:55.112850432   pavement_Vb.003                Background
406 2022-11-14 14:27:55.413427712     terrain_O.001                Background
422 2022-11-14 14:27:56.129650944          Crane_59           Global_Landmark
428 2022-11-14 14:27:56.318132480          cyclone0                Background
452 2022-11-14 14:27:56.685668352  CollisionObject1                Background
467 2022-11-14 14:27:56.960948736  CollisionObject1                Background
470 2022-11-14 14:27:57.244165376           crane_1                Background
483 2022-11-14 14:27:57.455461376           crane_1                Background
508 2022-11-14 14:27:57.888470528          cyclone0                Background
556 2022-11-14 14:27:58.532279040   TaskBuilding_35  TaskBuilding_Residential

In [7]:
data_Reduced.Collider_CategoricalN.unique()

array(['Background', 'TaskBuilding_Public', 'Building',
       'TaskBuilding_Residential', 'Active_Agent', 'Global_Landmark',
       'Active_Agent_Face', 'Passive_Agent', 'Passive_Agent_Face'],
      dtype=object)

In [8]:
entropy_df.head()

Filename  Overall_Transition_Entropy  Transition_Entropy_Background  \
0   1031_1                    0.389249                       0.335206   
1   1031_2                    0.378364                       0.317822   
2   1031_3                    0.442325                       0.376536   
3   1031_4                    0.471595                       0.378546   
4   1031_5                    0.391307                       0.314300   

   Stationary_Entropy_Background  Transition_Entropy_TaskBuilding_Public  \
0                       0.139582                                0.475571   
1                       0.124550                                0.511491   
2                       0.154199                                0.556428   
3                       0.149628                                0.694692   
4                       0.122028                                0.635757   

   Stationary_Entropy_TaskBuilding_Public  Transition_Entropy_Building  \
0                                0.090454                     0.438203   
1                                0.070284                     0.464861   
2                                0.080450                     0.436753   
3                                0.079991                     0.464841   
4                                0.057363                     0.471721   

   Stationary_Entropy_Building  Transition_Entropy_TaskBuilding_Residential  \
0                     0.152295                                     0.506264   
1                     0.146222                                     0.560104   
2                     0.153293                                     0.645078   
3                     0.139937                                     0.735646   
4                     0.131888                                     0.650421   

   Stationary_Entropy_TaskBuilding_Residential  \
0                                     0.069779   
1                                     0.071933   
2                                     0.087746   
3                                     0.076085   
4                                     0.071145   

   Transition_Entropy_Active_Agent  Stationary_Entropy_Active_Agent  \
0                         0.681446                         0.025486   
1                         0.657728                         0.021338   
2                         0.579004                         0.063003   
3                         0.584689                         0.069430   
4                         0.526395                         0.053301   

   Transition_Entropy_Global_Landmark  Stationary_Entropy_Global_Landmark  \
0                            0.455617                            0.037203   
1                            0.410918                            0.024851   
2                            0.478353                            0.018390   
3                            0.560158                            0.020855   
4                            0.460003                            0.019106   

   Transition_Entropy_Active_Agent_Face  Stationary_Entropy_Active_Agent_Face  \
0                              0.456680                              0.012404   
1                              0.509265                              0.006166   
2                              0.515783                              0.045352   
3                              0.635429                              0.041567   
4                              0.494206                              0.015423   

   Transition_Entropy_Passive_Agent  Stationary_Entropy_Passive_Agent  \
0                          0.828389                          0.013998   
1                          0.682804                          0.015484   
2                          0.682909                          0.027405   
3                          0.675854                          0.050550   
4                          0.730833                          0.035660   

   Transition_Entropy_Passive_Agent_Face  \
0                        

In [9]:
entropy_df.Overall_Transition_Entropy.describe()

count    145.000000
mean       0.398591
std        0.044607
min        0.284654
25%        0.375118
50%        0.397409
75%        0.432131
max        0.491865
Name: Overall_Transition_Entropy, dtype: float64

In [10]:
trials_df.head()

NameError: name 'trials_df' is not defined